In [2]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

load_smooth_dev()


ℹ Loading smooth
Exports from /home/filtheo/smooth/src/adamGeneral.cpp:

Exports from /home/filtheo/smooth/src/matrixPowerWrap.cpp:
   RcppExport SEXP matrixPowerWrap(SEXP matA, SEXP power)

Exports from /home/filtheo/smooth/src/ssGeneral.cpp:
   RcppExport SEXP initparams(SEXP Etype, SEXP Ttype, SEXP Stype, SEXP datafreq, SEXP obsR, SEXP obsallR, SEXP yt, SEXP damped, SEXP phi, SEXP smoothingparameters, SEXP initialstates, SEXP seasonalcoefs)
   RcppExport SEXP etsmatrices(SEXP matvt, SEXP vecg, SEXP phi, SEXP Cvalues, SEXP ncomponentsR, SEXP lagsModel, SEXP fittertype, SEXP Ttype, SEXP Stype, SEXP nexovars, SEXP matat, SEXP estimpersistence, SEXP estimphi, SEXP estiminit, SEXP estiminitseason, SEXP estimxreg, SEXP matFX, SEXP vecgX, SEXP gowild, SEXP estimFX, SEXP estimgX, SEXP estiminitX)
   RcppExport SEXP polysoswrap(SEXP ARorders, SEXP MAorders, SEXP Iorders, SEXP ARIMAlags, SEXP nComp, SEXP AR, SEXP MA, SEXP constant, SEXP Cvalues, SEXP matvt, SEXP vecg, SEXP matF, SEXP fitterty

R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.6 loaded.


R[write to console]: This is package "smooth", v4.3.2.41002


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package loaded in development mode


# Two-Stage Initialization Tests - R Implementation

Two-stage initialization works by:
1. First running a model with `initial="two-stage"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods


### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting


In [3]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [4]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with optimal initial:
Parameters: 0.02502502 98.50918 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134
[10] 101.134 101.134 101.134


In [5]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with backcasting (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [6]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'complete')
cat('ETS(A,N,N) with complete (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with complete (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [7]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with two-stage initialization (two-stage):
Parameters: 0 99.61911 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911


### Test 2: Local trend ETS(A,A,N)


In [8]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [19]:
%%R -i ts_df

model_twostage <- adam(ts_df, model = "AAN", lags = c(12), initial = 'two-stage', silent = TRUE)
# Check verbose output of a two-stage model
model_twostage$optim

NULL


In [10]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'complete')
cat('ETS(A,A,N) with complete initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] 


ETS(A,A,N) with complete initial:
Parameters: 0.1981801 0 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2483 165.8745 166.5008 167.1270 167.7532 168.3795 169.0057 169.6320
 [9] 170.2582 170.8844 171.5107 172.1369

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [11]:
%%R -i ts_df

model_complete <- adam(ts_df, model = "AAN", lags = c(12), initial = 'complete', silent = TRUE)
cat('R complete B:', model_complete$B, '\n')

R complete B: 0.1981801 0 


In [12]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,N) with two-stage initialization (two-stage):
Parameters: 0.452232 0 56.57685 -0.0006313486 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.4290 165.4283 165.4277 165.4271 165.4264 165.4258 165.4252 165.4245
 [9] 165.4239 165.4233 165.4226 165.4220

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 3: Seasonal data ETS(A,A,A)


In [11]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [12]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,A,A) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: 0.1262427 0.00535913 0.003895228 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 146.1119 158.7196 172.5692 204.7922 126.5985 164.4469 153.0418 150.9850
 [9] 240.4869 182.6033 136.5388 206.7759

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [13]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(A,A,A) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: 0.0136677 0.0136335 0.04507698 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 138.4338 152.1570 172.5888 207.9025 124.0372 163.2517 158.8099 152.2698
 [9] 246.2531 189.0066 139.0215 209.2510

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [14]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(A,A,A) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: 0.0136677 0.0136335 0.04507698 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 138.4338 152.1570 172.5888 207.9025 124.0372 163.2517 158.8099 152.2698
 [9] 246.2531 189.0066 139.0215 209.2510

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [15]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,A) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.02696647 0.0249973 0.02980314 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 146.3250 163.6461 181.4108 212.0395 135.3428 175.1023 166.3438 161.4340
 [9] 254.4911 195.5746 151.5578 223.9894

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 4: Damped trend ETS(A,Ad,N)


In [16]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'optimal')
cat('ETS(A,Ad,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with optimal initial:
Parameters: 0 0 0.9453906 214.7991 -2.425532 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 172.8556 172.8530 172.8506 172.8483 172.8461 172.8441 172.8422 172.8403
 [9] 172.8386 172.8370 172.8354 172.8339

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End =

 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [17]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'backcasting')
cat('ETS(A,Ad,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with backcasting initial:
Parameters: 0.01211439 0.002545899 0.9895939 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 172.3109 172.3690 172.4264 172.4833 172.5395 172.5952 172.6503 172.7048
 [9] 172.7587 172.8121 172.8649 172.9172

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [18]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,Ad,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with two-stage initialization (two-stage):
Parameters: 0.004251964 0.004173062 0.9495346 1513.036 -3.42304 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 171.3690 171.3331 171.2990 171.2666 171.2359 171.2067 171.1790 171.1527
 [9] 171.1277 171.1040 171.0815 171.0601

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 5: Multiplicative error ETS(M,N,N)


In [19]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with optimal initial:
Parameters: 0.03293295 189.757 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 


 [1] 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432
 [9] 174.2432 174.2432 174.2432 174.2432

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [20]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,N,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with backcasting initial:
Parameters: 0.03312988 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972
 [9] 174.1972 174.1972 174.1972 174.1972

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [21]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with two-stage initialization (two-stage):
Parameters: 0.03298676 1060.222 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.239 174.239 174.239 174.239 174.239 174.239 174.239 174.239 174.239
[10] 174.239 174.239 174.239

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 6: Multiplicative seasonal ETS(M,A,M)


In [22]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,M) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: 0.1144016 0.002874298 0.0009683954 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 141.9179 159.2557 176.0219 201.6552 126.9609 165.3692 155.2566 150.9177
 [9] 235.2195 178.3706 136.5630 206.6802

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [23]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,A,M) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: 0.01224624 0.01217939 0 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 141.5050 149.1423 177.3594 212.1294 123.9787 157.5993 162.9417 152.4868
 [9] 249.4937 197.9690 133.6372 192.6964

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [24]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(M,A,M) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: 0.01224624 0.01217939 0 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 141.5050 149.1423 177.3594 212.1294 123.9787 157.5993 162.9417 152.4868
 [9] 249.4937 197.9690 133.6372 192.6964

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [25]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,A,M) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.02153054 0.02076881 0.1811976 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 147.3522 165.5316 177.0305 208.7956 134.4075 176.6699 165.0942 161.2446
 [9] 256.0138 195.1860 154.1551 227.2916

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



## Summary

Two-stage initialization successfully:
- Runs a two-stage backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches
